# **Exploration of COVIDActNow data**

---

### *An introductory data collection and analysis project on COVID-19 data sourced from COVIDActNow.org*


Tara Ahuja, Jonathan Arp, Jesse Delacruz Jr., Derrick Ngo

---

The following code pulls data from the API and creates our initial dataframe and .csv file that each individual references for analysis.

In [2]:
#import dependencies
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from config import covid_api

In [ ]:
#setup base url and perform initial get request
covid_base_url = 'https://api.covidactnow.org/v2/states.json?apiKey='+covid_api
covid_json = requests.get(covid_base_url).json()

In [ ]:
#create dataframe using concat
states_df = pd.DataFrame()
for state in covid_json:
    state_temp = pd.DataFrame({
        'State':[state['state']],
        'Population':[state['population']],
        
        'Vaccination Ratio':[state['metrics']['vaccinationsInitiatedRatio']],
        'Weekly New Cases (per 100k)':[state['metrics']['weeklyNewCasesPer100k']],
        'Infection Rate':[state['metrics']['infectionRate']],
        
        'Cases':[state['actuals']['cases']],
        'Deaths':[state['actuals']['deaths']],
        'Hospital Beds':[state['actuals']['hospitalBeds']['capacity']],
        'Covid ICU Usage':[state['actuals']['hospitalBeds']['currentUsageCovid']]
    })
    #add new row
    states_df=pd.concat([
        states_df,
        state_temp
    ], ignore_index=True)

In [ ]:
#export dataframe to csv
states_df.to_csv('StatesCleaned.csv')

### **The following code is Jonathan's work to answer this question:...**

### **The following code is Derrick's work to answer this question:...**

In [ ]:
#import data from StatesCleaned.csv and drop extra index column
states_data = pd.read_csv('StatesCleaned.csv')
states_data = states_data.drop('Unnamed: 0',axis=1)

In [ ]:
#create a function to prompt for user input
def prompt():
    #ask user for which two states to compare
    state1 = input("Enter the two-letter abbreviation of the first state you'd like to compare: ")
    state2 = input("Enter the two-letter abbreviation of the second state you'd like to compare: ")

    #create list of selectable metrics (from the dataframe columns)
    metrics_list = ''
    index = 1
    for metric_name in states_data.columns[2:]:
        metrics_list = metrics_list+str(index)+': '+str(metric_name)+',\n' 
        index+=1
        
    #ask user for desired comparison metric
    metric_index = input(f"Select the metric (type the number) you'd like to compare {state1} and {state2} on: "+metrics_list)

    #return user input as a list
    return [state1,state2,metric_index]

In [ ]:
#prompt user and check for input errors
while True:
    try:
        inputs = prompt()
        #check if metric index is within bounds
        column_name = states_data.columns[int(inputs[2])+1]
        #boolean evaluation for correct state abbreviations
        state_check = (states_data.loc[states_data['State']==inputs[0]].any().any()) and (states_data.loc[states_data['State']==inputs[1]].any().any())
        if ~state_check:
            raise Exception
    except:
        #if any input is incorrectly formatted (not two-letter abbreviation, or not a possible column number), retry input
        print('Those inputs were not accepted. Retrying... ')
    else:
        #if inputs are acceptable, move on
        print('Inputs accepted...')
        break

This code creates the desired visualization from the user's input. After taking two state abbreviations and a selected metric (column) from the user, a graph comparing the stats for that metric is created.

In [ ]:
#access the requested data in the dataframe and present it in a dual bar graph
x = [inputs[0], inputs[1]]
y = [states_data.loc[states_data['State']==inputs[0]][column_name].values[0],states_data.loc[states_data['State']==inputs[1]][column_name].values[0]]

#create visualization, set attributes, save image, and display
plt.figure(facecolor='white')
plt.bar(x,y,edgecolor='black')

plt.xlabel('States')
plt.ylabel(column_name)
initial_yticks = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(x) for x in initial_yticks])
plt.title(f'{column_name} for {inputs[0]} and {inputs[1]}')

plt.savefig('singlecomparison.png')

plt.show()

This next block takes the same states from the earlier input, and plots ALL of the metrics
This block achieves this using a single loop, standardizing the plots. This results in a less-than-ideal representation/scaling of the graphs side-by-side, but completes this process with less code than individualized/specific data parsing.

In [ ]:
#compare ALL metrics for chosen states
x = [inputs[0], inputs[1]]
column_index = 2

#create visualization and set attributes
tuple_axs = ('ax1','ax2','ax3','ax4','ax5','ax6','ax7')
fig, tuple_axs = plt.subplots(1,7, sharex = True)
fig.set_facecolor('white')
fig.suptitle(f'Comparison between {inputs[0]} and {inputs[1]}', fontsize=16, fontweight='bold')
fig.set_size_inches(35,5)

# loop through all metrics and create a standardized plot for each
for ax in tuple_axs:
    #retrieve data for each metric
    column_name = states_data.columns[column_index]
    y = [states_data.loc[states_data['State']==inputs[0]][column_name].values[0],states_data.loc[states_data['State']==inputs[1]][column_name].values[0]]
    
    #create plot, set attributes
    ax.bar(x,y,edgecolor='black', color = ['sandybrown','dodgerblue'])
    ax.set_ylabel = states_data.columns[column_index]
    initial_yticks = ax.get_yticks()
    ax.set_yticklabels(['{:,.1f}'.format(x) for x in initial_yticks])
    ax.set_title(f'{column_name}')

    #increment index tracker
    column_index += 1

#save and display graphic
fig.tight_layout()

plt.savefig('allmetrics.png')
plt.show()

### **The following code is Jesse's work to answer this question:...**

### **The following code is Tara's work to answer this question:...**